In [592]:
#Import dependencies
import pandas as pd
import re

# Dependencies for MongoDB
import pymongo
from pprint import pprint  # For json output easy to read

# Dependency to convert dataframe to json format for MongoDB
import json

import requests
import os

import sys

from bs4 import BeautifulSoup as bs

In [593]:
url = "https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor"
url

'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor'

In [594]:
res = requests.get(url)
res

<Response [200]>

In [595]:
soup = bs(res.text,'html.parser')
#soup

In [596]:
table = soup.find(class_= "wikitable sortable").find("tbody")
#table

In [598]:
year_data = table.find_all(scope = "row")

In [599]:
#l = table.find_all(style="background:#FAEB86;")
#m = l[0].text.strip()
#print(m)

In [600]:
back_ground_yellow = table.find_all(style="background:#FAEB86;")
len(back_ground_yellow)

279

In [605]:
#back_ground_yellow
#if(back_ground_yellow[153].find("small")):
 #   print("b")
    
#else:
 #   print("c")

In [614]:
actors_list =[]
films_list = []
years_list = []
tie_index = []

for i in (range(len(year_data))):
    years_list.append(year_data[i].text.strip())  # total number of unique years are added to years_list

for i in (range(len(back_ground_yellow))):
    #print(i)
    if (i%3 == 0): # All first column elemnets in the table(actor names) are added to actors_list
        
        actor = back_ground_yellow[i].find_all("b")[0].find("span").find("a").text
        actors_list.append(actor)
     
        ind = int(i/3)
        # print(ind)
        
        if(back_ground_yellow[i].find("small")):  # if "Tie-up" winners are found, add indexes to "tie_index"
            #print(ind)
            #print("d")
            tie_index.append(ind)  
            if (len(tie_index)==2):  # Assuming 2 "Tie-up" winners in a year
                if (tie_index[-1]-1) == tie_index[-2]:
                    years_list.insert(tie_index[-1],years_list[tie_index[-2]])  #insert the "Year" value for the second "Tie-up" winner
                    tie_index.clear()  # Clear the list for the next scenario of "Tie-up" winners 
                     

    if (i%3 == 2): # All third column elemnets in the table(Film names) are added to films_list
        
        films_list.append(back_ground_yellow[i].find("b").text)
 

In [615]:
print(len(actors_list))
print(len(films_list))
print(len(years_list))

93
93
93


In [626]:
actor_df = pd.DataFrame({"Years": years_list, "WinnersName": actors_list, "Film": films_list,"Category":"Best Actor"})
actor_df.head(10)

,Years,WinnersName,Film,Category
0,1927/28 (1st),Emil Jannings,The Last CommandThe Way of All Flesh,Best Actor
1,1928/29(2nd)[note 1],Warner Baxter,In Old Arizona,Best Actor
2,1929/30 (3rd),George Arliss,Disraeli,Best Actor
3,1930/31 (4th),Lionel Barrymore,A Free Soul,Best Actor
4,1931/32 (5th),Wallace Beery,The Champ,Best Actor
5,1931/32 (5th),Fredric March,Dr. Jekyll and Mr. Hyde,Best Actor
6,1932/33 (6th),Charles Laughton,The Private Life of Henry VIII,Best Actor
7,1934 (7th),Clark Gable,It Happened One Night,Best Actor
8,1935 (8th),Victor McLaglen,The Informer,Best Actor
9,1936 (9th),Paul Muni,The Story of Louis Pasteur,Best Actor


In [625]:
#bestactor_df.head(10)

In [629]:
actor_url = []
for i in actors_list:
    #print(i)
    actor_url.append((f"https://en.wikipedia.org/wiki/{i}"))

In [633]:
actor_df["Wiki URL"] = actor_url
actor_df.head(10)

,Years,WinnersName,Film,Category,Wiki URL
0,1927/28 (1st),Emil Jannings,The Last CommandThe Way of All Flesh,Best Actor,https://en.wikipedia.org/wiki/Emil Jannings
1,1928/29(2nd)[note 1],Warner Baxter,In Old Arizona,Best Actor,https://en.wikipedia.org/wiki/Warner Baxter
2,1929/30 (3rd),George Arliss,Disraeli,Best Actor,https://en.wikipedia.org/wiki/George Arliss
3,1930/31 (4th),Lionel Barrymore,A Free Soul,Best Actor,https://en.wikipedia.org/wiki/Lionel Barrymore
4,1931/32 (5th),Wallace Beery,The Champ,Best Actor,https://en.wikipedia.org/wiki/Wallace Beery
5,1931/32 (5th),Fredric March,Dr. Jekyll and Mr. Hyde,Best Actor,https://en.wikipedia.org/wiki/Fredric March
6,1932/33 (6th),Charles Laughton,The Private Life of Henry VIII,Best Actor,https://en.wikipedia.org/wiki/Charles Laughton
7,1934 (7th),Clark Gable,It Happened One Night,Best Actor,https://en.wikipedia.org/wiki/Clark Gable
8,1935 (8th),Victor McLaglen,The Informer,Best Actor,https://en.wikipedia.org/wiki/Victor McLaglen
9,1936 (9th),Paul Muni,The Story of Louis Pasteur,Best Actor,https://en.wikipedia.org/wiki/Paul Muni


In [ ]:
birthplace =[]
birthyear = []


for url in actor_url:
    response = requests.get(url)
    soup = bs(response.text,'html.parser')
    #print(soup)
    #birthplace_result = soup.find_all("div",class_="birthplace")[0].text
    #print(birthplace_result)
    #birthplace.append(birthplace_result)
    
    try:
        birthplace_result = soup.find_all("div",class_="birthplace")[0].text
        birthplace.append(birthplace_result)
        birthyear_result = soup.find_all('span', class_='bday')[0].text[:4]
        birthyear.append(birthyear_result)
    #except IndexError as I:
    except:
        #print(I)
        print("Oops!",sys.exc_info()[0],"occured")
        birthplace.append("Unknown")
        birthyear.append("Unknown")
        

Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured


In [ ]:
actor_df["Birthplace"] = birthplace
actor_df["Birth Year"] = birthyear

In [ ]:
actor_df.head(10)

In [617]:
#Import raw data from webpage
raw_df = pd.read_html("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor")[2]
raw_df.head(20)

,Year,Actor,Role(s),Film,Ref.
0,1927/28 (1st),Emil Jannings [A],Grand Duke Sergius AlexanderAugust Schilling,The Last CommandThe Way of All Flesh,[10]
1,1927/28 (1st),Richard Barthelmess,Nickie ElkinsPatent Leather Kid,The NooseThe Patent Leather Kid,[10]
2,1928/29(2nd)[note 1],Warner Baxter,The Cisco Kid,In Old Arizona,[11]
3,1928/29(2nd)[note 1],George Bancroft,Thunderbolt Jim Lang,Thunderbolt,[11]
4,1928/29(2nd)[note 1],Chester Morris,Chick Williams,Alibi,[11]
5,1928/29(2nd)[note 1],Paul Muni,James Dyke,The Valiant,[11]
6,1928/29(2nd)[note 1],Lewis Stone,Count Pahlen,The Patriot,[11]
7,1929/30 (3rd),George Arliss [B],Benjamin Disraeli,Disraeli,[12]
8,1929/30 (3rd),George Arliss[B],The Raja,The Green Goddess,[12]
9,1929/30 (3rd),Wallace Beery,"Butch ""Machine Gun"" Schmidt",The Big House,[12]


In [618]:
tie_index = []
unique_index = []

# this is for "Tie-up" data , In 5th year , 2 actors won the Academy Award 

for i in range(len(raw_df)):
    if (raw_df["Actor"][i].find("(TIE)")==-1):
        unique_index.append(i)
    else:
        tie_index.append(i)
       
    
tie_index

[18, 19]

In [619]:
year_unique_tie = []
actor_unique_tie = []
film_unique_tie = []
m=0
#Loop through the data to select only the Best Actor winners in each year. 
#Best Actor is always listed first in each Oscar year amongst the nominees.

for i in (range(len(raw_df))):  
    #print(i)
    if i in tie_index:
        #print(i)
        year_unique_tie.append(raw_df['Year'][i])
        actor_unique_tie.append(raw_df['Actor'][i])
        film_unique_tie.append(raw_df['Film'][i])
        
    else:
        #print(i)
        if raw_df['Year'][i] not in year_unique_tie:
            year_unique_tie.append(raw_df['Year'][i])
            actor_unique_tie.append(raw_df['Actor'][i])
            film_unique_tie.append(raw_df['Film'][i]) 
    
len(actor_unique_tie)  

93

In [622]:
bestactor_df = pd.DataFrame({"Years": year_unique_tie, "Winner Name":actor_unique_tie, "Film": film_unique_tie ,"Category":"Best Actor"})
bestactor_df.head(10)

,Years,Winner Name,Film,Category
0,1927/28 (1st),Emil Jannings [A],The Last CommandThe Way of All Flesh,Best Actor
1,1928/29(2nd)[note 1],Warner Baxter,In Old Arizona,Best Actor
2,1929/30 (3rd),George Arliss [B],Disraeli,Best Actor
3,1930/31 (4th),Lionel Barrymore,A Free Soul,Best Actor
4,1931/32 (5th),Wallace Beery (TIE) [C],The Champ,Best Actor
5,1931/32 (5th),Fredric March (TIE) [C],Dr. Jekyll and Mr. Hyde,Best Actor
6,1932/33 (6th),Charles Laughton,The Private Life of Henry VIII,Best Actor
7,1934 (7th),Clark Gable,It Happened One Night,Best Actor
8,1935 (8th),Victor McLaglen,The Informer,Best Actor
9,1936 (9th),Paul Muni,The Story of Louis Pasteur,Best Actor


In [56]:
winnernames = [bestactor_df["Winner Name"]]
for n in range(len(winnernames[0])):
    #print(winnernames[0][n])
    actor_unique_tie[n]=re.sub("[\(\[].*?[\)\]]","",winnernames[0][n])
    
#winnernames[0]
type(actor_unique_tie)

list

In [57]:
actor_url = []
for i in actor_unique_tie:
    #print(i)
    actor_url.append((f"https://en.wikipedia.org/wiki/{i}"))

In [58]:
#actor_url

In [59]:
#Create List of unique oscar years
#list_of_years = raw_df.Year.unique()
#list_of_years
#actor_url

In [60]:
bestactor_df["Wiki URL"] =actor_url

In [61]:
#Loop through the data to select only the Best Actor winners in each year. 
#Best Actor is always listed first in each Oscar year amongst the nominees.
#years = []
#actor=[]
#film=[]

#for x in list_of_years:
    #years.append(raw_df[raw_df["Year"]==x].iloc[0]["Year"])
    #actor.append(raw_df[raw_df["Year"]==x].iloc[0]["Actor"])
    #film.append(raw_df[raw_df["Year"]==x].iloc[0]["Film"])

In [64]:
bestactor_df.head(10)
#actor

,Years,Winner Name,Film,Category,Wiki URL
0,1927/28 (1st),Emil Jannings,The Last CommandThe Way of All Flesh,Best Actor,https://en.wikipedia.org/wiki/Emil Jannings
1,1928/29(2nd)[note 1],Warner Baxter,In Old Arizona,Best Actor,https://en.wikipedia.org/wiki/Warner Baxter
2,1929/30 (3rd),George Arliss,Disraeli,Best Actor,https://en.wikipedia.org/wiki/George Arliss
3,1930/31 (4th),Lionel Barrymore,A Free Soul,Best Actor,https://en.wikipedia.org/wiki/Lionel Barrymore
4,1931/32 (5th),Wallace Beery,The Champ,Best Actor,https://en.wikipedia.org/wiki/Wallace Beery
5,1931/32 (5th),Fredric March,Dr. Jekyll and Mr. Hyde,Best Actor,https://en.wikipedia.org/wiki/Fredric March
6,1932/33 (6th),Charles Laughton,The Private Life of Henry VIII,Best Actor,https://en.wikipedia.org/wiki/Charles Laughton
7,1934 (7th),Clark Gable,It Happened One Night,Best Actor,https://en.wikipedia.org/wiki/Clark Gable
8,1935 (8th),Victor McLaglen,The Informer,Best Actor,https://en.wikipedia.org/wiki/Victor McLaglen
9,1936 (9th),Paul Muni,The Story of Louis Pasteur,Best Actor,https://en.wikipedia.org/wiki/Paul Muni


In [40]:
birthplace =[]
birthyear = []


for url in actor_url:
    response = requests.get(url)
    soup = bs(response.text,'html.parser')
    #print(soup)
    #birthplace_result = soup.find_all("div",class_="birthplace")[0].text
    #print(birthplace_result)
    #birthplace.append(birthplace_result)
    
    try:
        birthplace_result = soup.find_all("div",class_="birthplace")[0].text
        birthplace.append(birthplace_result)
        birthyear_result = soup.find_all('span', class_='bday')[0].text[:4]
        birthyear.append(birthyear_result)
    #except IndexError as I:
    except:
        #print(I)
        print("Oops!",sys.exc_info()[0],"occured")
        birthplace.append("Unknown")
        birthyear.append("Unknown")
        

Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured


In [65]:
bestactor_df["Birthplace"] = birthplace
bestactor_df["Birth Year"] = birthyear

In [66]:
bestactor_df.head(10)

,Years,Winner Name,Film,Category,Wiki URL,Birthplace,Birth Year
0,1927/28 (1st),Emil Jannings,The Last CommandThe Way of All Flesh,Best Actor,https://en.wikipedia.org/wiki/Emil Jannings,"Rorschach, Switzerland",1884
1,1928/29(2nd)[note 1],Warner Baxter,In Old Arizona,Best Actor,https://en.wikipedia.org/wiki/Warner Baxter,"Columbus, Ohio, U.S.",1889
2,1929/30 (3rd),George Arliss,Disraeli,Best Actor,https://en.wikipedia.org/wiki/George Arliss,"London, England, UK",1868
3,1930/31 (4th),Lionel Barrymore,A Free Soul,Best Actor,https://en.wikipedia.org/wiki/Lionel Barrymore,"Philadelphia, Pennsylvania, U.S.",1878
4,1931/32 (5th),Wallace Beery,The Champ,Best Actor,https://en.wikipedia.org/wiki/Wallace Beery,"Clay County, Missouri, U.S.",1885
5,1931/32 (5th),Fredric March,Dr. Jekyll and Mr. Hyde,Best Actor,https://en.wikipedia.org/wiki/Fredric March,"Racine, Wisconsin, U.S.",1897
6,1932/33 (6th),Charles Laughton,The Private Life of Henry VIII,Best Actor,https://en.wikipedia.org/wiki/Charles Laughton,"Scarborough, North Riding of Yorkshire, England",1899
7,1934 (7th),Clark Gable,It Happened One Night,Best Actor,https://en.wikipedia.org/wiki/Clark Gable,"Cadiz, Ohio, U.S.",1901
8,1935 (8th),Victor McLaglen,The Informer,Best Actor,https://en.wikipedia.org/wiki/Victor McLaglen,"Stepney, East London, England, UK",1886
9,1936 (9th),Paul Muni,The Story of Louis Pasteur,Best Actor,https://en.wikipedia.org/wiki/Paul Muni,"Lemberg, Austro-Hungarian Empire (present day ...",1895


In [41]:
#Some of the actor names have end notes listed on Wikipedia next to their name - ex. [A]
#Some actors have tied for best actor. 
#I was able to remove the end notes from the names, but uncertain as to what to do about actors tying the award in the same year.
#Actors who tied for the award will be listed on a different line that was not pulled.
#Also need to remove § special charactor after some names.
#Tried re function with no sucess. Don't think it really applies to this.

#actors = pd.Series(actor).str.split("[").str[0].tolist()


In [429]:
#actors
#bestactor_df = pd.DataFrame({"Years": years, "Winner Name":actor, "Film": film,"Category":"Best Actor"})
#bestactor_df.head(20)

In [347]:
#year_unique_tie

In [430]:
# Connection for Mongod DB server

conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [56]:
# Database - "winnersdb" to store collection(table) of Documents(rows) and fields(columns)

database = client.winnersdb
database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'winnersdb')

In [60]:
#winners_collection(table) is to store all winners data from Concatenated dataframe

collection = database.winners_collection

In [431]:
# Database and collection are created at this point(if they dont exist) after we insert data 

# inserting Dataframe to MongoDB Collection


df_to_dict = json.loads(bestactor_df.T.to_json()).values()
#df_to_dict

In [78]:
collection.insert_many(df_to_dict)